In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from prop_confidence_intervals import wald

%matplotlib inline
sns.set(style="whitegrid", context="paper")

import warnings

warnings.filterwarnings("ignore")

 Read in & filter PiP dataset

In [ ]:
data = pd.read_csv("pip_cleaned_data.csv")
data = data[data.keep_sample == 1]  # Removing unpaired/erroneous samples

### Sample counts

In [ ]:
# Ensure the dataframe is sorted by id and container
df = data.sort_values(by=["Specimen Number", "container"])[
    [
        "Specimen Number",
        "All Small Particles",
        "Bacteria",
        "RBC (Urine)",
        "WBC (Urine)",
        "container",
    ]
].reset_index(drop=1)

In [ ]:
df.info()  # Show data types/ missing values

Missing microscopy results

In [ ]:
missing_data = df[df.isna().any(axis=1)]
missing_data

Drop missing specimens from data

In [ ]:
df = df[~df["Specimen Number"].isin(missing_data["Specimen Number"])]
df

Restructure data

In [ ]:
# Melt the dataframe to long format
df_melted = df.melt(
    id_vars=["container", "Specimen Number"],
    value_vars=["All Small Particles", "Bacteria", "RBC (Urine)", "WBC (Urine)"],
    var_name="count type",
    value_name="count value",
)
df_melted["log_transform"] = np.log(df_melted["count value"] + 1)
df_melted.head()

Outliers

In [ ]:
df_melted[
    df_melted["Specimen Number"].isin(["C02329514", "C02366588"])
]  # Outlier specimens

Remove outliers from analysis

In [ ]:
df_melted = df_melted[~df_melted["Specimen Number"].isin(["C02329514", "C02366588"])]

Data summary

In [ ]:
df_melted.groupby(by=["container", "count type"])[["count value"]].describe()

Visualise distributions for counts

In [ ]:
# Create a 2 by 2 plot
fig, axs = plt.subplots(2, 2, figsize=(9, 7))

# Flatten the axs array for easy iteration
axs = axs.flatten()

for x, ax, ref in zip(
    ["All Small Particles", "Bacteria", "RBC (Urine)", "WBC (Urine)"], axs, ['A', 'B', 'C', 'D']
):

    sns.kdeplot(
        data=df_melted[df_melted["count type"] == x],
        x="count value",
        hue="container",
        ax=ax,
    )

    ax.set_title(x, fontsize=12)
    
    if ref in ['C', 'D']:
        ax.set_xlabel(r"Microscopy counts $(10^6/\text{L})$", fontsize=12)
    else:
        ax.set_xlabel("", fontsize=12)
    
    if ref in ['B', 'D']:
        ax.set_ylabel("", fontsize=12)
    else:    
        ax.set_ylabel("Density", fontsize=12)

    ax.tick_params(axis="both", which="major", labelsize=10)

    # Only keep legend in first plot
    if x != "All Small Particles":
        ax.get_legend().remove()
        
    ax.text(-0.1, 1.1, ref, transform=ax.transAxes, 
            size=15, weight='bold')
    
fig.suptitle(
    "Kernel Density Estimation: Comparing PiP and Control Microscopy Distributions",
    fontsize=14,
)
fig.tight_layout()

plt.savefig(
    "figures/comparing_microscopy_distributions.png", dpi=600, bbox_inches="tight"
)

fig.show()

Visualise distribution (applying log transformation)

In [ ]:
# Create a 2 by 2 plot
fig, axs = plt.subplots(2, 2, figsize=(9, 7))

# Flatten the axs array for easy iteration
axs = axs.flatten()

for x, ax, ref in zip(
    ["All Small Particles", "Bacteria", "RBC (Urine)", "WBC (Urine)"], axs, ['A', 'B', 'C', 'D']
):

    sns.kdeplot(
        data=df_melted[df_melted["count type"] == x],
        x="log_transform",
        hue="container",
        ax=ax,
    )

    ax.set_title(x, fontsize=12)
    
    if ref in ['C', 'D']:
        ax.set_xlabel(r"$\log_{10}(\text{Microscopy counts} + 1)$", fontsize=12)
    else:
        ax.set_xlabel("", fontsize=12)
    
    if ref in ['B', 'D']:
        ax.set_ylabel("", fontsize=12)
    else:    
        ax.set_ylabel("Density", fontsize=12)

    ax.tick_params(axis="both", which="major", labelsize=10)

    # Only keep legend in first plot
    if x != "All Small Particles":
        ax.get_legend().remove()
        
    ax.text(-0.1, 1.1, ref, transform=ax.transAxes, 
            size=15, weight='bold')
    
fig.suptitle(
    "Kernel Density Estimation: Comparing log transformed PiP and Control Microscopy Distributions",
    fontsize=14,
)
fig.tight_layout()

plt.savefig(
    "figures/comparing_log_transformed_microscopy_distributions.png",
    dpi=600,
    bbox_inches="tight",
)

fig.show()

Restructure data to analyse differences

In [ ]:
difference_df = df_melted.pivot_table(
    columns=["container"], index=["count type", "Specimen Number"], values="count value"
)
difference_df["pip-control"] = difference_df["PIP"] - difference_df["PLASTIC"]
difference_df = difference_df.reset_index()
difference_df.head()

Summary of differences

In [ ]:
difference_df.groupby(by="count type")[["pip-control"]].describe()

Lower 95% range microscopy difference

In [ ]:
difference_df.groupby(by="count type")[["pip-control"]].quantile(0.025)

Upper 95% range microscopy difference

In [ ]:
difference_df.groupby(by="count type")[["pip-control"]].quantile(0.975)

Calculate confidence interval for differences

In [ ]:
# Create a 2 by 2 plot
fig, axs = plt.subplots(2, 2, figsize=(9, 7))

# Flatten the axs array for easy iteration
axs = axs.flatten()

# Initialise empty dictionaries
comparing_data = {}
paired_t_test_conf = {}

# Initialise empty lists
organsism_list = []
lower_ci_list = []
upper_ci_list = []
diff_list = []
sample_size_list = []
std_err_list = []

for c, ax, ref in zip(
    ["WBC (Urine)", "RBC (Urine)", "All Small Particles", "Bacteria"], axs, ['A', 'B', 'C', 'D']
):

    # Filter differences_df and drop instances of missing values
    filtered_differences_df = difference_df[difference_df["count type"] == c]

    # Obtain PiP and control Pandas.Series
    PIP_vals = filtered_differences_df["PIP"]
    PLASTIC_vals = filtered_differences_df["PLASTIC"]

    # Run a paired t-test (SciPy)
    t_test = ttest_rel(PIP_vals.values, PLASTIC_vals.values, alternative="two-sided")

    # Manually calculate mean difference
    mean_diff = filtered_differences_df["pip-control"].mean()

    organsism_list.append(c)

    # Obtain outputs from t_test object
    lower_ci_list.append(t_test.confidence_interval().low)
    upper_ci_list.append(t_test.confidence_interval().high)
    std_err_list.append(t_test._standard_error)
    diff_list.append(mean_diff)
    sample_size_list.append(t_test.df + 1)

    # Populate figure
    sns.kdeplot(data=filtered_differences_df, x="pip-control", ax=ax)

    ax.set_title(f"{c}", fontsize=12)
    if ref in ['C', 'D']:
        ax.set_xlabel(r"Microscopy count difference $(10^6/\text{L})$", fontsize=12)
    else:
        ax.set_xlabel("", fontsize=12)
    if ref in ['B', 'D']:
        ax.set_ylabel("", fontsize=12)
    else:    
        ax.set_ylabel("Density", fontsize=12)

    # Add confidence interval information
    confidence_interval = f"[{t_test.confidence_interval().low:.2f}, {t_test.confidence_interval().high:.2f}]"
    info_text = f"Mean Difference: {mean_diff:.2f}\n95% CI: {confidence_interval}"
    ax.text(
        0.05,
        0.95,
        info_text,
        transform=ax.transAxes,
        fontsize=10,
        verticalalignment="top",
        horizontalalignment="left",
        bbox=dict(facecolor="white", alpha=1),
    )

    ax.tick_params(axis="both", which="major", labelsize=10)

    ax.text(-0.1, 1.1, ref, transform=ax.transAxes, 
            size=15, weight='bold')
    
counts_diff = pd.DataFrame()
counts_diff["organism"] = organsism_list
counts_diff["difference"] = diff_list
counts_diff["sem"] = std_err_list
counts_diff["lower_ci"] = lower_ci_list
counts_diff["upper_ci"] = upper_ci_list
counts_diff["sample_size"] = sample_size_list

fig.suptitle(
    "Kernel Density Estimation: Distribution of Microscopy Count Differences (PiP - control)",
    fontsize=14,
)
fig.tight_layout()

plt.savefig("figures/microscopy_differences.png", dpi=600, bbox_inches="tight")

Summary table for differences

In [ ]:
counts_diff

Bland-Altman plot to compare differences

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

# Flatten the axs array for easy iteration
axs = axs.flatten()

for c, ax, ref in zip(
    ["WBC (Urine)", "RBC (Urine)", "All Small Particles", "Bacteria"], axs, ['A', 'B', 'C', 'D']
):

    # Filter differences_df and drop instances of missing values
    filtered_differences_df = difference_df[difference_df["count type"] == c]

    # Obtain PiP and control Pandas.Series
    PIP_vals = filtered_differences_df["PIP"]
    PLASTIC_vals = filtered_differences_df["PLASTIC"]
    
    sm.graphics.mean_diff_plot(PIP_vals, PLASTIC_vals, ax = ax)
    ax.set_title(f"{c}", fontsize=14)
    if ref in ['C', 'D']:
        ax.set_xlabel(r"Average of PiP & control", fontsize=12)
    else:
        ax.set_xlabel("", fontsize=12)

    if ref in ['B', 'D']:
        ax.set_ylabel("", fontsize=12)
    else:    
        ax.set_ylabel("Difference (pip - control)", fontsize=12)
    
    ax.text(-0.1, 1.1, ref, transform=ax.transAxes, 
            size=15, weight='bold')
    ax.set_xscale('log')
fig.suptitle(
    "Bland-Altman Plot: Agreement of Microscopy Counts",
    fontsize=18,
)
fig.tight_layout()

plt.savefig("figures/microscopy_bland_altman_log_scale.png", dpi=600, bbox_inches="tight")

### Culture growth

Mixed growth

In [ ]:
data_new = data[data.keep_sample == 1]  # Removing unpaired samples

In [ ]:
data_new["Comment Desc"] = (
    data_new["Comment Desc"].str.strip().str.lower().str.title()
)  # tidy strings

In [ ]:
data_new["Comment Desc"].value_counts()  # show counts for each comment

Mixed growth contingency table

In [ ]:
table = pd.crosstab(
    index=data_new[data_new.container == "PLASTIC"]["Mixed growth"].reset_index(
        drop=True
    ),
    columns=data_new[data_new.container == "PIP"]["Mixed growth"].reset_index(
        drop=True
    ),
    rownames=["Present in control"],
    colnames=["Present in PIP"],
).reindex(columns=[1, 0], index=[1, 0], fill_value=0)

display(table)

table.to_csv("tables/mixed_growth_contingency.csv")

Strep. B detection ***outside*** Antenatal

In [ ]:
data_non_antenatal_keep = data[(data.keep_sample == 1) & (data.ward != "ANTENATAL")]

In [ ]:
contingency_tbls = {}
for x in ["Streptococcus Group B"]:

    table = pd.crosstab(
        index=data_non_antenatal_keep[data_non_antenatal_keep.container == "PLASTIC"][
            x
        ].reset_index(drop=True),
        columns=data_non_antenatal_keep[data_non_antenatal_keep.container == "PIP"][
            x
        ].reset_index(drop=True),
        rownames=["Present in control"],
        colnames=["Present in PIP"],
    ).reindex(columns=[1, 0], index=[1, 0], fill_value=0)
    contingency_tbls[x] = table
    display(table)

table.to_csv("tables/strep_b_non_antenatal.csv")

Agreement in meeting culture thresholds

In [ ]:
def culture_asp(row):
    if row['All Small Particles'] > 10_000:
        return 1
    else:
        return 0
def culture_wbc(row):
    if row['WBC (Urine)'] > 45:
        return 1
    else:
        return 0
def culture_bacteria(row):
    if row['Bacteria'] > 5:
        return 1
    else:
        return 0
def culture_antenatal(row):
    if row['ward'] == 'ANTENATAL':
        return 1
    else:
        return 0
def culture_outside_antenatal(row):
    if row['ward'] == 'ANTENATAL':
        return np.nan
    elif row['cultured'] == 1:
        return 1
    else:
        return 0

Overall agreement in culture

In [ ]:
data['culture_from_asp'] = data.apply(culture_asp, axis=1)
data['culture_from_wbc'] = data.apply(culture_wbc, axis=1)
data['culture_from_bacteria'] = data.apply(culture_bacteria, axis=1)
data['culture_outside_antenatal'] = data.apply(culture_outside_antenatal, axis=1)

data['culture_threshold_passed'] = data[['culture_from_asp', 
                                         'culture_from_wbc', 
                                         'culture_from_bacteria']].any(axis=1).astype(int)

In [ ]:
target_vars = ['culture_from_asp', 'culture_from_wbc', 'culture_from_bacteria', 'cultured', 'culture_outside_antenatal']

In [ ]:
contingency_tbls_dict = {}
for x in target_vars:

    table = pd.crosstab(
        index=data[data.container == "PLASTIC"][x].reset_index(drop=True),
        columns=data[data.container == "PIP"][x].reset_index(drop=True),
        rownames=["Present in plastic container"],
        colnames=["Present in PIP container"],
    ).reindex(columns=[1, 0], index=[1, 0], fill_value=0)
    contingency_tbls_dict[x] = table

In [ ]:
combined = pd.concat(contingency_tbls_dict.values(), keys=contingency_tbls_dict.keys())
combined.index.names = ["Microscopy count", ""]
combined.to_csv("tables/microscopy_comparison_contengency.csv")
combined

In [ ]:
# Populate summary table
organsism_list = []
lower_ci_list = []
upper_ci_list = []
diff_list = []
pip_proportion_list = []
pla_proportion_list = []

final_table = []

for o in target_vars:

    table_o = contingency_tbls_dict[o]
    A = table_o.loc[1, 1]
    B = table_o.loc[0, 1]
    C = table_o.loc[1, 0]
    D = table_o.loc[0, 0]
    N = table_o.sum().sum()

    diff, lower_ci, upper_ci = wald(A, B, C, D, N)

    organsism_list.append(o)
    lower_ci_list.append(lower_ci)
    upper_ci_list.append(upper_ci)
    diff_list.append(diff)
    pip_proportion_list.append((A + B) / N)
    pla_proportion_list.append((A + C) / N)

    primary_outcomes_df = pd.DataFrame(index=[o])
    primary_outcomes_df["Cultured (PiP)"] = A + B
    primary_outcomes_df["Cultured (control)"] = A + C
    primary_outcomes_df['Difference in proportion'] = (
                                A + B
                            ) / N - (A + C) / N
    primary_outcomes_df["lower CI"] = round(
        lower_ci, 4
    )
    primary_outcomes_df["Upper CI"] = round(
        upper_ci, 4
    )

    final_table.append(primary_outcomes_df)

In [ ]:
summary_data = pd.concat(final_table)
summary_data.to_csv("tables/microscopy_detection_summary.csv")
summary_data